In [4]:
!pip install jupyter_http_over_ws
!jupyter serverextension enable --py jupyter_http_over_ws

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Enabling: jupyter_http_over_ws
- Writing config: /root/.jupyter
    - Validating...
      jupyter_http_over_ws 0.0.7 OK


In [5]:
!pip install DeepSaki
import DeepSaki


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
strategy, RUNTIME_ENVIRONMENT, hw_accelerator_handle = DeepSaki.utils.DetectHw()

Running on single GPU  /device:GPU:0
Number of accelerators:  1
____________________________________________________________________________________
Device List: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1285180698776677867
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1660524955
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15272102322374183360
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
